In [1]:
import os

In [2]:
%pwd

'c:\\Users\\parva\\OneDrive\\Desktop\\MOdel BUilding\\Chicken_Dissease\\chick_Dis_Class\\research'

In [3]:
# file opens in research folder since it is in research folder, change it 
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\parva\\OneDrive\\Desktop\\MOdel BUilding\\Chicken_Dissease\\chick_Dis_Class'

In [13]:
# lets create a data class i.e Update the entity from the flow
# it would be a return type for the function

# DataIngestionConfig is framed to store the locations and the str, for when we load the data as zip file
# this is what we will require
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir : Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [7]:
# now defining configuration manager
# this will define the configuration as where to store what based on config.yaml and params.yaml file
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH
    ):
       self.config = read_yaml(config_filepath) # returned as box type so artifacts_root can be accessed using .
       self.params = read_yaml(params_filepath)
       create_directories([self.config.artifacts_root])

    # it will return a an entty i.e custom return that is defined above as dataclasss { say object}
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion  # get data ingestion from config.yaml file

        create_directories([config.root_dir])     # create the root directory mentioned in yam.config

        data_ingestion_config = DataIngestionConfig (         # create an entity {object} from the provided configuration i.e config.yaml
            root_dir= config.root_dir,              # keep root_dir as mentioned in config.yaml, similarly others
            source_url= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config            # so when called it will create the directory mentioned in config.yaml and return and object
                                                # i.e entity with all the required info i.e root, source url, local dir, unzip folder 

In [8]:
# now components

In [9]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [14]:
# class with all the functions to perform the download and extract of the data
# provided the DataIngestionConfig object such that it contains paths to required directories
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following data \n {headers}")
        else:
            logger.info(f"file name already exists with size:{get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
# pipe line

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-09-21 11:40:45,988: INFO: common: yaml file: config\config.yaml loaded successfully]


[2024-09-21 11:40:45,990: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-21 11:40:45,994: INFO: common: created directory at: artifacts]
[2024-09-21 11:40:45,996: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-21 11:40:50,116: INFO: 2944298941: artifacts/data_ingestion/data.zip downloaded with following data 
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3609:B5E5C:284E54:32820D:66EE6369
Accept-Ranges: bytes
Date: Sat, 21 Sep 2024 06:10:52 GMT
Via: 1.1 varnish
X-Served-By: cache-del21730-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1726899051.314486,VS0,VE1182
Vary: Authorization,Accept-Enco

In [15]:
# data ingestion in ipynb is done and working fine, now we need to setup it as modular code
# for that again follow the steps in readmeMD to update for modular code